In [29]:
import sys
from awsglue.context import GlueContext
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from pyspark.sql import functions as F
from pyspark.sql.types import BooleanType
from pyspark.ml.feature import Tokenizer

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
logger = glueContext.get_logger()



In [34]:
output_path_predictions = "s3://nequijmmarinq/predictions/"
model_output_path = "s3://nequijmmarinq/models/msg-classifier"
test_path = "s3://nequijmmarinq/test/"



In [32]:
try:
    from pyspark.ml import PipelineModel
    logger.info("Cargando modelo desde S3...")
    model = PipelineModel.load(model_output_path)
    logger.info("Realizando inferencia en batch...")
    df = spark.read.parquet(test_path)
    predictions = model.transform(df)
    predictions.select("body", "prediction", "msg_length").write.mode("overwrite").parquet(output_path_predictions)
    logger.info("Inferencia completada y resultados guardados.")

except Exception as e:
    logger.warn(f"Modelo no encontrado o error en inferencia: {str(e)}")

logger.info("Pipeline completo.")


In [37]:
from pyspark.ml import PipelineModel
model = PipelineModel.load(model_output_path)
logger.info("Cargando modelo desde S3...")


In [38]:
df = spark.read.parquet(test_path)

In [39]:
df.count()

563505


In [40]:
predictions = model.transform(df)

In [41]:
predictions.show()

+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|inbound|          created_at|                text|              tokens|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|      0|Fri Aug 11 11:20:...|735053 definitely...|[735053, definite...|(10000,[733,2405,...|(10000,[733,2405,...|[4.50687417574296...|[0.98908750319478...|       0.0|
|      0|Fri Dec 01 00:00:...|739582 may tight ...|[739582, may, tig...|(10000,[1815,2170...|(10000,[1815,2170...|[1.43749055694009...|[0.80806574898097...|       0.0|
|      0|Fri Dec 01 05:29:...|741557 awaiting r...|[741557, awaiting...|(10000,[182,1147,...|(10000,[182,1147,...|[4.15077030939504...|[0.98449200850567...|    